In [ ]:
from Modules import *
import datetime
import time


def download_with_date_range(ticker, start_date, end_date, interval="1d"):
    """
    Download stock data with specific date range
    
    Parameters:
    ticker (str): Stock ticker symbol
    start_date (str): Start date in format 'YYYY-MM-DD'
    end_date (str): End date in format 'YYYY-MM-DD'
    interval (str): Data interval
    
    Returns:
    pandas.DataFrame: Stock data
    """
    try:
        stock = yf.Ticker(ticker)
        data = stock.history(start=start_date, end=end_date, interval=interval)
        data = data.reset_index()
        data = data.drop(columns=['Dividends','Stock Splits'],axis=1)
        return data
    except Exception as e:
        print(f"Error downloading data: {e}")
        return None
    

def wwma(values, n):
    """
     J. Welles Wilder's EMA 
    """
    return values.ewm(alpha=1/n, adjust=False).mean()

def atr(data, n=20):
    # data = df.copy()
    high = data["High"]
    low = data["Low"]
    close = data["Close"]
    data['tr0'] = abs(high - low)
    data['tr1'] = abs(high - close.shift())
    data['tr2'] = abs(low - close.shift())
    tr = data[['tr0', 'tr1', 'tr2']].max(axis=1)
    atr = wwma(tr, n)
    data['atr'] = atr
    
    data = data.drop(columns=['tr0','tr1','tr2'], axis=1)
    
    return data

munch is not present in the environment. See https://trading-ig.readthedocs.io/en/latest/faq.html#optional-dependencies
can't import config from config file


Download historical data

In [ ]:
listOfStocks = ["IBIT","XLK","XLF","XLV","XLY","XLI","XLC","XLP","XLE","XLRE","XLB","XLU"]

for z in range(len(listOfStocks)):
    data = download_with_date_range(listOfStocks[z],'1990-01-01','2025-12-31','1d')
    atr(data)

    data['return'] = np.log(data['Close']/data['Close'].shift(1))
    data.dropna(inplace=True)
    print(listOfStocks[z]+": "+str(data['return'].std()))

# data

IBIT: 0.031693649526479746


Turtle Trading Code 

In [ ]:
# Parameters for the Turtle Trading Strategy
S1_EntryWindow = 20
S1_ExitWindow = 10
S2_EntryWindow = 55
S2_ExitWindow = 20

position_size = 1  # Number of units to trade (simplified for this example)
stop_loss_multiplier = 2  # Multiplier for the ATR to set stop loss

# Initialize columns for signals and positions
data['Signal'] = 0
data['Position'] = 0
data['StopLoss'] = 0


# Calculate entry and exit breakouts for S1
data['High_20'] = data['High'].rolling(window=S1_EntryWindow).max()
data['Low_20'] = data['Low'].rolling(window=S1_EntryWindow).min()

data['High_10'] = data['High'].rolling(window=S1_ExitWindow).max()
data['Low_10'] = data['Low'].rolling(window=S1_ExitWindow).min()

data['EntryLong_S1'] = np.where(data['High'].shift(1) >= data['High_20'].shift(1),1,0)
data['EntryShort_S1'] = np.where(data['Low'].shift(1) <= data['Low_20'].shift(1),1,0)    

data['ExitLong_S1'] = np.where(data['Low'] <= data['Low_10'],1,0)
data['ExitShort_S1'] = np.where(data['High'] >= data['High_10'],1,0)
 
data.dropna(inplace=True)
data.tail(3)



,Date,Open,High,Low,Close,Volume,Capital Gains,tr0,tr1,tr2,...,Low_10,Signal_int,Position,StopLoss,Low_20,High_10,EntryLong_S1,EntryShort_S1,ExitLong_S1,ExitShort_S1
449,2025-10-27 00:00:00-04:00,65.375000,65.760002,65.000000,65.279999,49298000,0.0,0.760002,2.930000,2.169998,...,59.310001,0,0,0,59.310001,65.760002,0,0,0,1
450,2025-10-28 00:00:00-04:00,65.349998,65.970001,64.410004,64.489998,47609800,0.0,1.559998,0.690002,0.869995,...,59.310001,0,0,0,59.310001,65.970001,0,0,0,1
451,2025-10-29 00:00:00-04:00,64.485001,64.510002,62.000000,62.750000,76452400,0.0,2.510002,0.020004,2.489998,...,59.310001,0,0,0,59.310001,65.970001,0,0,0,0


In [58]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

class TurtleTrading:
    """
    Implementation of the Turtle Trading System with:
    - Entry signals (20/55 day breakouts)
    - Stop loss (2 ATR)
    - Pyramiding (up to 4 units)
    - Position sizing based on account risk
    """
    
    def __init__(self, capital=100000, risk_per_trade=0.01, atr_period=20,
                 entry_period=20, exit_period=10, max_units=4):
        """
        Initialize Turtle Trading System
        
        Parameters:
        - capital: Starting capital
        - risk_per_trade: Risk per unit (1% = 0.01)
        - atr_period: Period for ATR calculation
        - entry_period: Breakout period for entry (20 or 55 days)
        - exit_period: Breakout period for exit (10 or 20 days)
        - max_units: Maximum units to pyramid (typically 4)
        """
        self.initial_capital = capital
        self.capital = capital
        self.risk_per_trade = risk_per_trade
        self.atr_period = atr_period
        self.entry_period = entry_period
        self.exit_period = exit_period
        self.max_units = max_units
        
        self.position = 0  # Current position size
        self.entry_prices = []  # Track entry prices for pyramiding
        self.stop_loss = 0
        self.trades = []
        
    def calculate_atr(self, df):
        """Calculate Average True Range"""
        high = df['High']
        low = df['Low']
        close = df['Close']
        
        tr1 = high - low
        tr2 = abs(high - close.shift())
        tr3 = abs(low - close.shift())
        
        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        atr = tr.rolling(window=self.atr_period).mean()
        
        return atr
    
    def calculate_position_size(self, price, atr):
        """
        Calculate position size based on risk
        Dollar volatility = ATR * dollars per point
        Position size = (Account * Risk%) / (2 * ATR)
        """
        if atr == 0 or pd.isna(atr):
            return 0
        
        dollar_volatility = atr
        position_size = (self.capital * self.risk_per_trade) / (2 * dollar_volatility)
        
        return int(position_size)
    
    def check_entry_signal(self, df, idx):
        """Check for breakout entry signal"""
        if idx < self.entry_period:
            return False
        
        current_price = df.loc[idx, 'High']
        lookback = df.loc[idx-self.entry_period:idx-1, 'High']
        
        # Entry when price breaks above N-day high
        if current_price > lookback.max():
            return True
        
        return False
    
    def check_exit_signal(self, df, idx):
        """Check for breakout exit signal"""
        if idx < self.exit_period:
            return False
        
        current_price = df.loc[idx, 'Low']
        lookback = df.loc[idx-self.exit_period:idx-1, 'Low']
        
        # Exit when price breaks below N-day low
        if current_price < lookback.min():
            return True
        
        return False
    
    def check_pyramid_signal(self, current_price, last_entry_price, atr):
        """
        Check if we can add another unit (pyramid)
        Add unit when price moves 0.5 ATR in favorable direction
        """
        if len(self.entry_prices) >= self.max_units:
            return False
        
        if current_price >= last_entry_price + (0.5 * atr):
            return True
        
        return False
    
    def update_stop_loss(self, atr):
        """Update stop loss to 2 ATR below the last entry price"""
        if len(self.entry_prices) > 0:
            self.stop_loss = self.entry_prices[-1] - (2 * atr)
    
    def run_backtest(self, df):
        """
        Run the Turtle Trading backtest
        
        Parameters:
        - df: DataFrame with columns ['Date', 'Open', 'High', 'Low', 'Close']
        """
        df = df.copy()
        df['ATR'] = self.calculate_atr(df)
        
        equity_curve = []
        
        for idx in df.index:
            current_date = df.loc[idx, 'Date']
            current_price = df.loc[idx, 'Close']
            atr = df.loc[idx, 'ATR']
            
            if pd.isna(atr):
                equity_curve.append(self.capital)
                continue
            
            # Check stop loss
            if self.position > 0 and current_price <= self.stop_loss:
                pnl = sum([(current_price - entry) * self.position / len(self.entry_prices) 
                          for entry in self.entry_prices])
                self.capital += pnl
                
                self.trades.append({
                    'Date': current_date,
                    'Type': 'Stop Loss',
                    'Price': current_price,
                    'Position': -self.position,
                    'PnL': pnl,
                    'Capital': self.capital
                })
                
                self.position = 0
                self.entry_prices = []
                self.stop_loss = 0
            
            # Check exit signal
            elif self.position > 0 and self.check_exit_signal(df, idx):
                pnl = sum([(current_price - entry) * self.position / len(self.entry_prices) 
                          for entry in self.entry_prices])
                self.capital += pnl
                
                self.trades.append({
                    'Date': current_date,
                    'Type': 'Exit',
                    'Price': current_price,
                    'Position': -self.position,
                    'PnL': pnl,
                    'Capital': self.capital
                })
                
                self.position = 0
                self.entry_prices = []
                self.stop_loss = 0
            
            # Check pyramid signal (add units)
            elif self.position > 0 and len(self.entry_prices) < self.max_units:
                if self.check_pyramid_signal(current_price, self.entry_prices[-1], atr):
                    unit_size = self.calculate_position_size(current_price, atr)
                    self.position += unit_size
                    self.entry_prices.append(current_price)
                    self.update_stop_loss(atr)
                    
                    self.trades.append({
                        'Date': current_date,
                        'Type': f'Pyramid Unit {len(self.entry_prices)}',
                        'Price': current_price,
                        'Position': unit_size,
                        'PnL': 0,
                        'Capital': self.capital
                    })
            
            # Check entry signal
            elif self.position == 0 and self.check_entry_signal(df, idx):
                entry_price = df.loc[idx, 'High']
                unit_size = self.calculate_position_size(entry_price, atr)
                
                if unit_size > 0:
                    self.position = unit_size
                    self.entry_prices = [entry_price]
                    self.update_stop_loss(atr)
                    
                    self.trades.append({
                        'Date': current_date,
                        'Type': 'Entry Unit 1',
                        'Price': entry_price,
                        'Position': unit_size,
                        'PnL': 0,
                        'Capital': self.capital
                    })
            
            # Track unrealized PnL
            if self.position > 0:
                unrealized_pnl = sum([(current_price - entry) * self.position / len(self.entry_prices) 
                                     for entry in self.entry_prices])
                equity_curve.append(self.capital + unrealized_pnl)
            else:
                equity_curve.append(self.capital)
        
        return pd.DataFrame(self.trades), equity_curve
    
    def get_performance_metrics(self, equity_curve):
        """Calculate performance metrics"""
        returns = pd.Series(equity_curve).pct_change().dropna()
        
        total_return = (equity_curve[-1] - self.initial_capital) / self.initial_capital
        sharpe_ratio = returns.mean() / returns.std() * np.sqrt(252) if returns.std() != 0 else 0
        max_drawdown = (pd.Series(equity_curve) / pd.Series(equity_curve).cummax() - 1).min()
        
        winning_trades = [t for t in self.trades if t['PnL'] > 0]
        losing_trades = [t for t in self.trades if t['PnL'] < 0]
        
        win_rate = len(winning_trades) / len([t for t in self.trades if t['PnL'] != 0]) if self.trades else 0
        
        return {
            'Total Return': f"{total_return:.2%}",
            'Final Capital': f"${equity_curve[-1]:,.2f}",
            'Sharpe Ratio': f"{sharpe_ratio:.2f}",
            'Max Drawdown': f"{max_drawdown:.2%}",
            'Total Trades': len([t for t in self.trades if 'Unit 1' in t['Type']]),
            'Win Rate': f"{win_rate:.2%}"
        }


# Example usage
if __name__ == "__main__":
    # Generate sample data
    np.random.seed(42)
    # dates = pd.date_range(start='1990-01-01', end='2024-01-01', freq='D')
    
    df = download_with_date_range('GLD','2000-01-01','2025-12-31','1d')

    # # Simulate price data with trend
    # close = 100 + np.cumsum(np.random.randn(len(dates)) * 2)
    # high = close + np.random.uniform(0, 2, len(dates))
    # low = close - np.random.uniform(0, 2, len(dates))
    # open_price = close + np.random.uniform(-1, 1, len(dates))
    
    # df = pd.DataFrame({
    #     'Date': dates,
    #     'Open': open_price,
    #     'High': high,
    #     'Low': low,
    #     'Close': close
    # })
    
    # Initialize and run backtest
    turtle = TurtleTrading(
        capital=100000,
        risk_per_trade=0.01,
        entry_period=20,
        exit_period=10,
        max_units=4
    )
    
    trades_df, equity_curve = turtle.run_backtest(df)
    
    # Display results
    print("=== TURTLE TRADING BACKTEST RESULTS ===\n")
    print("Performance Metrics:")
    metrics = turtle.get_performance_metrics(equity_curve)
    for key, value in metrics.items():
        print(f"{key}: {value}")
    
    print("\n=== Recent Trades ===")
    print(trades_df.tail(10).to_string(index=False))
    
    print(f"\n=== Position Details ===")
    print(f"Max Units: {turtle.max_units}")
    print(f"Risk per Trade: {turtle.risk_per_trade:.1%}")
    print(f"ATR Period: {turtle.atr_period}")
    print(f"Entry Period: {turtle.entry_period} days")
    print(f"Exit Period: {turtle.exit_period} days")

=== TURTLE TRADING BACKTEST RESULTS ===

Performance Metrics:
Total Return: 266.87%
Final Capital: $366,866.71
Sharpe Ratio: 0.50
Max Drawdown: -22.24%
Total Trades: 122
Win Rate: 27.87%

=== Recent Trades ===
                     Date           Type      Price  Position          PnL       Capital
2025-06-13 00:00:00-04:00 Pyramid Unit 2 316.290009       321     0.000000 284788.247693
2025-06-24 00:00:00-04:00      Stop Loss 306.190002      -644 -5477.223145 279311.024549
2025-07-21 00:00:00-04:00   Entry Unit 1 313.309998       395     0.000000 279311.024549
2025-07-22 00:00:00-04:00 Pyramid Unit 2 316.100006       391     0.000000 279311.024549
2025-07-25 00:00:00-04:00      Stop Loss 307.399994      -786 -5741.736237 273569.288312
2025-08-28 00:00:00-04:00   Entry Unit 1 315.079987       521     0.000000 273569.288312
2025-08-29 00:00:00-04:00 Pyramid Unit 2 318.070007       554     0.000000 273569.288312
2025-09-02 00:00:00-04:00 Pyramid Unit 3 325.589996       501     0.000000 273

Filter stocks

Create list of Stocks

Import historic data    

In [4]:
listOfStocks = ['SPY','JNK','AGG',"XLU","VNQ","XLE","XLF","XLB","XLP","SPLV","IWO","QUAL","IWN","SPHD","IWM","TLT","MUB","TIP","BKLN","LQD","CWB","SHY"]
print("Number of Stocks: ",str(len(listOfStocks))) 

try:

    data = yf.download(
        tickers=listOfStocks,
        start='2000-01-01',
        end='2025-12-31',
        interval='1d',  # '1m', '5m', '1h', etc. for intraday (limited to recent data)
        group_by='ticker',  # Groups columns by ticker (alternative: 'column' for flat structure)
        threads=True,  # Enables multi-threading for faster downloads
        auto_adjust=True,  # Automatically adjusts for splits/dividends
        prepost=True  # Includes pre/post-market data
    )

    data.dropna(inplace=True)
    data.reset_index(inplace=True)
    data.to_csv("data.csv")
    data = pd.read_csv("data.csv")
    data = data.drop(index=0)
    data = data.drop(index=1)

    df_filtered = data.filter(like='.3')
    keepCols = df_filtered.columns.tolist()
    keepCols.insert(0,"Date")
    data = data[keepCols]


    for i in range(len(listOfStocks)):  
        data = data.rename(columns={listOfStocks[i]+'.3':listOfStocks[i]})
        data[listOfStocks[i]] = data[listOfStocks[i]].astype(float)
        data[listOfStocks[i]+'_ret'] = np.log(data[listOfStocks[i]]/(data[listOfStocks[i]].shift(1)))

except:
    "Not Found"

data.dropna(inplace=True)
df_filtered = data.filter(like='_ret')
keepCols = df_filtered.columns.tolist()
keepCols.insert(0,"Date")
data = data[keepCols]
data = data.set_index("Date")
display(data.tail(5))


# import requests
# from io import StringIO

# # Download Nominal GDP
# url = 'https://fred.stlouisfed.org/graph/fredgraph.csv?id=GDP'
# response = requests.get(url)
# df_gdp = pd.read_csv(StringIO(response.text))

# df_gdp = df_gdp.rename(columns={'observation_date':'Date'})
# df_gdp = df_gdp.set_index("Date")

# display(df_gdp.tail(1))


# # OUTER MERGE ON INDEX
# merged = pd.merge(
#     left=data,
#     right=df_gdp,
#     left_index=True,
#     right_index=True,
#     how='inner'          # <-- full outer join
# )

# # Download CPI (All Urban Consumers, Seasonally Adjusted)
# url = 'https://fred.stlouisfed.org/graph/fredgraph.csv?id=CPIAUCSL'
# response = requests.get(url)
# df_cpi = pd.read_csv(StringIO(response.text))
# df_cpi = df_cpi.rename(columns={'observation_date':'Date'})
# df_cpi = df_cpi.set_index("Date")

# # OUTER MERGE ON INDEX
# mergedInfl = pd.merge(
#     left=merged,
#     right=df_cpi,
#     left_index=True,
#     right_index=True,
#     how='inner'          # <-- full outer join
# )

# display(mergedInfl)



# Correlation
print("")
corr = data.corr()
# display(corr)
display(corr.style.background_gradient(cmap='coolwarm'))
print("")


Number of Stocks:  22


[*********************100%***********************]  22 of 22 completed


,SPY_ret,JNK_ret,AGG_ret,XLU_ret,VNQ_ret,XLE_ret,XLF_ret,XLB_ret,XLP_ret,SPLV_ret,...,IWN_ret,SPHD_ret,IWM_ret,TLT_ret,MUB_ret,TIP_ret,BKLN_ret,LQD_ret,CWB_ret,SHY_ret
Date,,,,,,,,,,,,,,,,,,,,,
2025-10-23,0.005912,0.001849,-0.001780,-0.000775,-0.000651,0.013238,0.002096,0.011930,-0.004903,-0.005910,...,0.008963,-0.005741,0.012578,-0.006867,-0.000651,-0.001963,0.000481,-0.001243,0.008877,-0.000722
2025-10-24,0.008139,0.002971,0.000989,0.011666,0.002925,-0.010167,0.010790,-0.004148,-0.003282,-0.001932,...,0.012195,-0.000206,0.012141,0.000437,0.000651,-0.000089,0.001920,0.001687,0.012793,0.000602
2025-10-27,0.011729,0.002861,0.000692,0.002732,0.002809,0.002382,0.004134,-0.000449,0.001642,0.003034,...,-0.001378,0.003900,0.003482,0.003383,0.000093,-0.000268,0.001917,0.001684,0.006229,-0.000241
2025-10-28,0.002652,-0.001123,0.000593,-0.016615,-0.019831,-0.010476,-0.006396,0.001348,-0.010022,-0.012053,...,-0.005366,-0.014445,-0.005368,0.002612,-0.000558,0.000804,0.000957,0.000089,-0.002465,0.000120
2025-10-29,0.000480,-0.002966,-0.004751,-0.000333,-0.026088,0.007413,-0.017130,-0.019839,-0.023871,-0.019989,...,-0.012783,-0.015923,-0.008552,-0.010158,-0.001396,-0.005280,-0.000478,-0.005684,0.004925,-0.001927


,SPY_ret,JNK_ret,AGG_ret,XLU_ret,VNQ_ret,XLE_ret,XLF_ret,XLB_ret,XLP_ret,SPLV_ret,IWO_ret,QUAL_ret,IWN_ret,SPHD_ret,IWM_ret,TLT_ret,MUB_ret,TIP_ret,BKLN_ret,LQD_ret,CWB_ret,SHY_ret
SPY_ret,1.000000,0.761977,0.083628,0.555914,0.720313,0.633118,0.853056,0.844690,0.708870,0.815986,0.863803,0.982204,0.824323,0.790537,0.867690,-0.193402,0.202027,0.019034,0.604425,0.256706,0.847495,-0.089336
JNK_ret,0.761977,1.000000,0.315374,0.518090,0.680103,0.543880,0.679027,0.698834,0.552951,0.675584,0.696872,0.750182,0.701660,0.694742,0.717944,0.015002,0.385630,0.209315,0.724865,0.496920,0.718083,0.120525
AGG_ret,0.083628,0.315374,1.000000,0.223161,0.270051,-0.015028,-0.026398,0.074274,0.118319,0.144017,0.110706,0.096530,0.085488,0.133163,0.096920,0.836598,0.660337,0.763373,0.088317,0.853569,0.161515,0.730950
XLU_ret,0.555914,0.518090,0.223161,1.000000,0.715596,0.361077,0.481684,0.510235,0.687050,0.811795,0.408688,0.546206,0.480812,0.741228,0.455339,0.051299,0.343244,0.160138,0.458085,0.324476,0.422725,0.123534
VNQ_ret,0.720313,0.680103,0.270051,0.715596,1.000000,0.483692,0.662156,0.670036,0.674598,0.814908,0.661136,0.715737,0.728108,0.831633,0.710950,0.035141,0.371728,0.196107,0.567850,0.389407,0.631394,0.110279
XLE_ret,0.633118,0.543880,-0.015028,0.361077,0.483692,1.000000,0.682102,0.685771,0.414218,0.524875,0.574852,0.606322,0.704140,0.699359,0.653522,-0.241958,0.107134,0.053395,0.480928,0.095090,0.545113,-0.147274
XLF_ret,0.853056,0.679027,-0.026398,0.481684,0.662156,0.682102,1.000000,0.815340,0.622086,0.756889,0.754166,0.829746,0.856346,0.822535,0.823948,-0.302905,0.125867,-0.073350,0.574175,0.139179,0.692213,-0.188202
XLB_ret,0.844690,0.698834,0.074274,0.510235,0.670036,0.685771,0.815340,1.000000,0.635137,0.752113,0.770327,0.833420,0.819571,0.808583,0.815660,-0.195152,0.174690,0.032354,0.565643,0.214078,0.724599,-0.073511
XLP_ret,0.708870,0.552951,0.118319,0.687050,0.674598,0.414218,0.622086,0.635137,1.000000,0.868843,0.512355,0.711936,0.562115,0.775656,0.550986,-0.096947,0.232850,0.029089,0.446401,0.229267,0.510585,0.010314
SPLV_ret,0.815986,0.675584,0.144017,0.811795,0.814908,0.524875,0.756889,0.752113,0.868843,1.000000,0.650016,0.810228,0.696237,0.872852,0.690215,-0.104433,0.299604,0.066563,0.581832,0.286396,0.644169,-0.002440


Turtle Trading